In [2]:
import pandas as pd

In [16]:
def setup(file = "ELO.csv"):
    return pd.read_csv( file)

In [7]:
def readScore( st):
    [A, B] = st.split(",")
    [teamA, scoreA] = A.split("-")
    [teamB, scoreB] = B.split("-")

    scoreA = int(scoreA)
    scoreB = int(scoreB)
    
    return [ (teamA,scoreA), (teamB,scoreB)]

In [4]:
def updateELO(rating, teamA, teamB, sA, sB):
    if (sA > sB):
        sA = 1
    elif( sA < sB):
        sA = 0
    else:
        sA = .5
    sB = 1 - sA
    
    qA = 10**(rating[teamA] / 400)
    qB = 10**(rating[teamB] / 400)
    
    eA = qA/(qA + qB)
    eB = qB/(qA + qB)
    
    rating[teamA] = rating[teamA] + 20 (sA - eA)
    rating[teamB] = rating[teamB] + 20 (sB - eB)
    
    return rating

In [6]:
flag = False

print("Please enter the name of the file containing new game scores to parse. \n  Enter \"DONE.\" to exit.")
file = input("Filename: ")

while( file != "DONE."):
    try:
        st = open(file)
        i = 0
        tA = ""
        
        for st in file
            if( tA is in st):
                i = i + 1
            else:
                i = 1
                
            [(tA,sA),(tB,sB)] = readScore( st)
            
            #control
            if( i == 1 or ):
                control = setup("ControlELO.csv")
            #hybrid    
            elif( i == 2):
                hybrid = setup("ELO.csv")
            #assault    
            elif( i == 3):
                 = setup("AssaultELO.csv")
            #escort
            else:
                escort = setup("Escort.csv")
    except FileNotFoundError:
        print("File not found. Please re-enter file name.\n")
        flag = True
    
    if( flag):
        flag = False
    else:
        print("File complete.")
        
    print( "Please enter the name of the file containing new game scores to parse. \n  Enter \"DONE.\" to exit.")
    file = input( "Filename: ")

Please enter the name of the file containing new game scores to parse. 
  Enter "DONE." to exit.
Filename: f
File not found. Please re-enter file name.

Please enter the name of the file containing new game scores to parse. 
  Enter "DONE." to exit.
Filename: DONE.


In [27]:
rating = setup()
other = rating
rating = pd.concat([rating, other]).reset_index()
del rating["index"]
d = dict(rating.iloc[-1,:])
d["BOS"] = 1000
d

{' BOS': 1500,
 ' CHD': 1500,
 ' DAL': 1500,
 ' FLA': 1500,
 ' GLA': 1500,
 ' GZC': 1500,
 ' HOU': 1500,
 ' HZS': 1500,
 ' LDN': 1500,
 ' NYE': 1500,
 ' PAR': 1500,
 ' PHI': 1500,
 ' SEO': 1500,
 ' SFS': 1500,
 ' SHD': 1500,
 ' TOR': 1500,
 ' VAL': 1500,
 ' VAN': 1500,
 ' WAS': 1500,
 'ATL': 1500,
 'BOS': 1000}